# IOM Evaluation Repository Overview
> Exploratory data analysis of `evaluations.json`

In [ ]:
#| echo: false
import pandas as pd
from fastcore.all import *
from pathlib import Path

In [ ]:
data = (Path('../files/test') / 'evaluations.json').read_json()
data[0]